In [146]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

In [147]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

In [148]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [149]:
# Import citipy
from citipy import citipy
# Import the datetime module from the datetime library.
from datetime import datetime
# Import the requests library.
import requests

In [150]:
# We create a cities list to store city names.
cities = []
# We iterate through the coordinates, as in our practice, and retrieve the nearest city using the latitude and longitude pair.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # We add a decision statement with the logical operator not in to determine whether the found city is already in the cities list. If not, then we'll use the append() function to add it. We are doing this because among the 1,500 latitudes and longitudes, there might be duplicates, which will retrieve duplicate cities, and we want to be sure we capture only the unique cities.
    if city not in cities:
        cities.append(city)

In [151]:
# Print the city count to confirm sufficient count.
len(cities)

747

In [152]:
# Import the API key.
from config import weather_api_key
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [153]:
# Create an empty list to hold the weather data.
city_data = []

print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval     
-----------------------------


In [154]:
# We create the for loop with the enumerate() method and reference the index and the city in the list.
for i, city in enumerate(cities):

    # In the conditional statement, we check if the remainder of the index divided by 50 is equal to 0
    # and if the index is greater than or equal to 50.
    if (i % 50 == 0 and i >= 50):
        # If the statement is true, then the set_count and the record_count are equal to 1.
        set_count += 1
        record_count = 1
    city_url = url + "&q=" + city.replace(" ","+")

    record_count += 1

    # TRY-EXCEPT BLOCKS (6.2.6)
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = str(city_weather["weather"][0]["description"])
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Description": city_description})

# If an error is experienced, skip the city.
    except:
        pass

In [155]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,Rocha,-34.48,-54.33,51.26,92,30,7.61,UY,scattered clouds
1,Comarapa,-17.90,-64.48,64.27,60,99,0.76,BO,overcast clouds
2,Tongliao,43.61,122.27,53.11,36,0,10.65,CN,clear sky
3,Bandarbeyla,9.49,50.81,76.91,78,5,1.30,SO,clear sky
4,Chokurdakh,70.63,147.92,21.74,93,94,6.02,RU,overcast clouds
5,New Norfolk,-42.78,147.06,66.00,51,87,1.99,AU,overcast clouds
6,Porto Novo,6.50,2.60,77.00,94,40,2.24,BJ,scattered clouds
7,Yar-Sale,66.83,70.83,38.77,90,51,9.46,RU,broken clouds
8,Severo-Kurilsk,50.68,156.12,48.72,69,2,1.72,RU,clear sky
9,Busselton,-33.65,115.33,72.00,46,2,2.48,AU,clear sky


In [156]:
# Adjust column order
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed","Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Rocha,UY,-34.48,-54.33,51.26,92,30,7.61,scattered clouds
1,Comarapa,BO,-17.90,-64.48,64.27,60,99,0.76,overcast clouds
2,Tongliao,CN,43.61,122.27,53.11,36,0,10.65,clear sky
3,Bandarbeyla,SO,9.49,50.81,76.91,78,5,1.30,clear sky
4,Chokurdakh,RU,70.63,147.92,21.74,93,94,6.02,overcast clouds
5,New Norfolk,AU,-42.78,147.06,66.00,51,87,1.99,overcast clouds
6,Porto Novo,BJ,6.50,2.60,77.00,94,40,2.24,scattered clouds
7,Yar-Sale,RU,66.83,70.83,38.77,90,51,9.46,broken clouds
8,Severo-Kurilsk,RU,50.68,156.12,48.72,69,2,1.72,clear sky
9,Busselton,AU,-33.65,115.33,72.00,46,2,2.48,clear sky


In [157]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")